In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
import json

In [2]:
!ls /kaggle/input

cote-divoire-byte-sized-agriculture-challenge
ctedivoire-byte-sizedagriculturechallengedataset


In [3]:
# Load training data
train = gpd.read_file('/kaggle/input/cote-divoire-byte-sized-agriculture-challenge/train.geojson')
print(f"Training samples: {len(train)}")
print("Crop types distribution:")
print(train['crop'].value_counts())

# Load test data
test = gpd.read_file('/kaggle/input/cote-divoire-byte-sized-agriculture-challenge/test.geojson')
print(f"\nTest samples: {len(test)}")

Training samples: 953
Crop types distribution:
crop
Rubber    405
Palm      313
Cocoa     235
Name: count, dtype: int64

Test samples: 282


In [4]:
# ## 2. Feature Engineering from Geometries

# %%
def extract_geometry_features(gdf):
    """Extract features from polygon geometries"""
    features = []

    for geom in gdf.geometry:
        # Handle both Polygon and MultiPolygon
        if geom.geom_type == 'MultiPolygon':
            areas = [p.area for p in geom.geoms]
            perims = [p.length for p in geom.geoms]
            poly = geom.geoms[0]  # Take largest polygon for other features
        else:
            areas = [geom.area]
            perims = [geom.length]
            poly = geom

        # Basic shape features
        area = sum(areas)
        perimeter = sum(perims)
        compactness = (4 * np.pi * area) / (perimeter ** 2)

        # Convex hull features
        convex_hull = poly.convex_hull
        hull_area = convex_hull.area
        solidity = area / hull_area if hull_area > 0 else 0

        # Bounding box features
        minx, miny, maxx, maxy = poly.bounds
        width = maxx - minx
        height = maxy - miny
        aspect_ratio = width / height if height > 0 else 0

        # Moment invariants
        coords = np.array(poly.exterior.coords)
        dx = coords[:,0] - minx
        dy = coords[:,1] - miny
        m00 = area
        m10 = np.sum(dx)
        m01 = np.sum(dy)
        centroid_x = m10 / m00
        centroid_y = m01 / m00
        mu20 = np.sum((dx - centroid_x)**2) / m00
        mu02 = np.sum((dy - centroid_y)**2) / m00
        mu11 = np.sum((dx - centroid_x)*(dy - centroid_y)) / m00

        features.append({
            'area': area,
            'perimeter': perimeter,
            'compactness': compactness,
            'solidity': solidity,
            'aspect_ratio': aspect_ratio,
            'num_polygons': len(areas),
            'largest_area': max(areas),
            'smallest_area': min(areas),
            'mean_area': np.mean(areas),
            'area_std': np.std(areas),
            'mu20': mu20,
            'mu02': mu02,
            'mu11': mu11,
            'width': width,
            'height': height
        })

    return pd.DataFrame(features)

# Extract features
train_features = extract_geometry_features(train)
test_features = extract_geometry_features(test)

# Add target
train_features['crop'] = train['crop'].values
train = train.drop('crop', axis=1)

In [5]:
X = train_features.drop('crop', axis=1)
y = train_features['crop']

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GEMINI_API_KEY")


In [7]:
from typing import List, Dict, Any, TypeVar, Callable, Optional
import functools
from tqdm import tqdm
from time import time, sleep
from datetime import datetime
# Type variable for generic function return
T = TypeVar('T')

# GeminiRateLimiter class (provided in paste.txt)
class GeminiRateLimiter:
    """
    A rate limiter for Google Gemini API that ensures requests don't exceed
    a specified number of requests per minute.
    
    This class implements a sliding window approach to track API calls and
    enforces waiting periods when necessary to stay within rate limits.
    """
    
    def __init__(self, requests_per_minute: int = 15, verbose: bool = True):
        """
        Initialize the rate limiter.
        
        Args:
            requests_per_minute: Maximum number of requests allowed per minute
            verbose: Whether to print status messages during rate limiting
        """
        self.requests_per_minute = requests_per_minute
        self.verbose = verbose
        self.request_times: List[float] = []  # Timestamps of recent requests
    
    def wait_if_needed(self):
        """
        Check if we need to wait before making another request and wait if necessary.
        This ensures we don't exceed the rate limit.
        """
        current_time = time()
        
        # Remove request times older than 60 seconds (sliding window)
        while self.request_times and current_time - self.request_times[0] > 60:
            self.request_times.pop(0)
        
        # If we're at the rate limit, wait until we can make another request
        if len(self.request_times) >= self.requests_per_minute:
            # Calculate when the oldest request will be 60+ seconds old
            wait_until = self.request_times[0] + 60  
            wait_time = wait_until - current_time
            
            if wait_time > 0:
                if self.verbose:
                    red_print(f"Rate limiting: Waiting {wait_time:.2f} seconds to stay within "
                          f"{self.requests_per_minute} requests/minute")
                sleep(wait_time)
                
                # After waiting, remove old request times again
                current_time = time()
                while self.request_times and current_time - self.request_times[0] > 60:
                    self.request_times.pop(0)
    
    def record_request(self):
        """
        Record that a request has been made.
        """
        self.request_times.append(time())
        if self.verbose:
            green_print(f"{datetime.now().strftime('%H:%M:%S')} - API request made "
                  f"({len(self.request_times)}/{self.requests_per_minute} in current window)")
    
    def execute(self, func: Callable[..., T], *args, **kwargs) -> T:
        """
        Execute a function with rate limiting.
        
        Args:
            func: The function to execute (typically a Gemini API call)
            *args: Positional arguments to pass to the function
            **kwargs: Keyword arguments to pass to the function
            
        Returns:
            The result of the function call
        """
        self.wait_if_needed()
        
        try:
            result = func(*args, **kwargs)
            self.record_request()
            return result
        except Exception as e:
            # Still record the request even if it failed
            # (API rate limits usually count failed requests too)
            self.record_request()
            raise e
    
    def __call__(self, func: Callable[..., T]) -> Callable[..., T]:
        """
        Decorator for rate-limiting a function.
        
        Usage:
            rate_limiter = GeminiRateLimiter(requests_per_minute=15)
            
            @rate_limiter
            def call_gemini_api(prompt):
                return client.generate_content(prompt)
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return self.execute(func, *args, **kwargs)
        return wrapper

def red_print(x):
    print(f"\033[91m{x}\033[0m")
def green_print(x):
    print(f"\033[92m{x}\033[0m")

In [8]:
rate_limiter = GeminiRateLimiter(requests_per_minute=10)

In [9]:
base_code = """
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
def extract_geometry_features(gdf):

    features = []
    
    for geom in gdf.geometry:
        # Handle both Polygon and MultiPolygon
        if geom.geom_type == 'MultiPolygon':
            areas = [p.area for p in geom.geoms]
            perims = [p.length for p in geom.geoms]
            poly = geom.geoms[0]  # Take largest polygon for other features
        else:
            areas = [geom.area]
            perims = [geom.length]
            poly = geom
    
        # Basic shape features
        area = sum(areas)
        perimeter = sum(perims)
        compactness = (4 * np.pi * area) / (perimeter ** 2)
    
        # Convex hull features
        convex_hull = poly.convex_hull
        hull_area = convex_hull.area
        solidity = area / hull_area if hull_area > 0 else 0
    
        # Bounding box features
        minx, miny, maxx, maxy = poly.bounds
        width = maxx - minx
        height = maxy - miny
        aspect_ratio = width / height if height > 0 else 0
    
        # Moment invariants
        coords = np.array(poly.exterior.coords)
        dx = coords[:,0] - minx
        dy = coords[:,1] - miny
        m00 = area
        m10 = np.sum(dx)
        m01 = np.sum(dy)
        centroid_x = m10 / m00
        centroid_y = m01 / m00
        mu20 = np.sum((dx - centroid_x)**2) / m00
        mu02 = np.sum((dy - centroid_y)**2) / m00
        mu11 = np.sum((dx - centroid_x)*(dy - centroid_y)) / m00
    
        features.append({
            'area': area,
            'perimeter': perimeter,
            'compactness': compactness,
            'solidity': solidity,
            'aspect_ratio': aspect_ratio,
            'num_polygons': len(areas),
            'largest_area': max(areas),
            'smallest_area': min(areas),
            'mean_area': np.mean(areas),
            'area_std': np.std(areas),
            'mu20': mu20,
            'mu02': mu02,
            'mu11': mu11,
            'width': width,
            'height': height
        })
    
    return pd.DataFrame(features)
"""

In [10]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
from google import genai
from google.genai import types

@rate_limiter
def generate(examples):
    client = genai.Client(
        api_key=user_secrets.get_secret("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"
    format_string = {'code': 'your generated code'}
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=f"""Generate a Python function for feature engineering on a geopandas GeoDataFrame. 
- Use geopandas, pandas, and shapely. 
- Please refer to example function below. 
- you must generate new features not existing in the example code.
- Handle potential errors gracefully. 
- Your generated python function must be named as add_new_features.
- The function must return a pandas dataframe with new features only. they must be numerical types.
- Please output the code directly in json format {format_string}

-- Start Example --

{examples}

-- End Example --"""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type="application/json",
    )

    res = []
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        #print(chunk.text, end="")
        res.append(chunk.text)
    return ''.join(res)

def feature_selection(X, y, test_features, test_ids, model, max_iter=3, margin=0.05):
    """Feature selection loop"""
    best_features = X.copy()
    examples = [base_code]
    print('Run baseline')
    best_submission, best_score = train_and_predict(X, y, test_features, test_ids, model)
    print(f'Best score:{best_score:.4f}')
    for i in range(max_iter):
        print(f"Iteration {i+1}/{max_iter}")
        try:
            res = generate('\n\n'.join(examples))
        except:
            print('generate new function failed')
            continue
        try:
            function_string  = json.loads(res.strip())['code']
        except:
            print('invalid function string')
            continue
        namespace = {}
        print(f'Generated new function:\n{function_string}')
        exec(function_string, namespace)
        
        # The function object 'greet' is now in the 'namespace' dictionary
        add_new_features = namespace['add_new_features']

        # Generate new features
        try:
            new_train_features = add_new_features(train)
            new_test_features = add_new_features(test)
    
            # Combine with existing features
            combined_train_features = pd.concat([best_features, new_train_features], axis=1)
            combined_test_features = pd.concat([test_features, new_test_features], axis=1)
    
            # Train and predict
            submission, score = train_and_predict(combined_train_features, y, combined_test_features, test_ids, model)
        except:
            print('Execution failed')
            continue

        # Check if the score improved meaningfully
        if score > best_score+margin:
            print(f"Score improved from {best_score:.4f} to {score:.4f}. Adding new features.")
            best_score = score
            best_features = combined_train_features
            examples.append(function_string)
            best_submission = submission
        # else:
        #     print(f"Score did not improve. Keeping the current features.")
        #     break  # Stop if the score doesn't improve

    return best_features, best_score, best_submission, examples

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [11]:
# prompt: rewrite the cell above as a function. it takes data and model, returns submission and f1 scores
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
def train_and_predict(X, y, test_features, test_ids, model):
    """
    Trains a model on the provided data and makes predictions.

    Args:
        data: A dictionary containing training and test data.
        model: The machine learning model to use.

    Returns:
        A tuple containing the submission DataFrame and the mean F1 score.
    """

    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    f1_scores = []
    fold_probabilities = []
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Fit and transform the target variable
    y_encoded = label_encoder.fit_transform(y)


    for fold, (train_index, val_index) in enumerate(skf.split(X, y_encoded)):
        print(f"Fold {fold+1}")
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y_encoded[train_index], y_encoded[val_index]

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold).astype(int)
        f1 = f1_score(y_val_fold, y_pred, average='weighted')
        f1_scores.append(f1)
        print(f"F1 Score: {f1}")

        fold_probs = model.predict_proba(test_features)
        fold_probabilities.append(fold_probs)

    mean_f1 = np.mean(f1_scores)
    print(f"\nMean F1 Score across 5 folds: {mean_f1}")

    average_probabilities = np.mean(fold_probabilities, axis=0)
    predicted_labels = np.argmax(average_probabilities, axis=1).astype(int)
    predicted_crop_types = label_encoder.inverse_transform(predicted_labels)
    submission = pd.DataFrame({'ID': test_ids, 'Target': predicted_crop_types})

    return submission, mean_f1


In [12]:
rf_classifier = RandomForestClassifier(random_state=42)
best_features, best_score, submission, best_code = feature_selection(X, y, test_features, test['ID'], 
                                                                     rf_classifier,
                                                                     max_iter=10, margin=0.2)

print(f"Best F1 Score: {best_score}")

Run baseline
Fold 1
F1 Score: 0.5377457167354148
Fold 2
F1 Score: 0.58152824153881
Fold 3
F1 Score: 0.6024019813134631
Fold 4
F1 Score: 0.6197024844853872
Fold 5
F1 Score: 0.5896506130190341

Mean F1 Score across 5 folds: 0.5862058074184218
Best score:0.5862
Iteration 1/10
12:13:02 - API request made (1/10 in current window)
Generated new function:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import transform
import pyproj

def add_new_features(gdf):
    """Adds new geometric features to a GeoDataFrame.

    Args:
        gdf (gpd.GeoDataFrame): The input GeoDataFrame.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the newly engineered features.
                      Returns an empty DataFrame if an error occurs.
    """
    features = []
    
    for index, row in gdf.iterrows():
        geom = row.geometry
        try:
            # Handle both Polygon and MultiPolygon
           

In [13]:
# Save
submission.to_csv('submission.csv', index=False)
print("\nSubmission saved:")
print(submission.head())


Submission saved:
          ID  Target
0  ID_UrUGR0    Palm
1  ID_3ZmbBi    Palm
2  ID_tPmH4c  Rubber
3  ID_rUfFQH  Rubber
4  ID_RrthDZ  Rubber


In [14]:
for func in best_code:
    print(func)


import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
def extract_geometry_features(gdf):

    features = []
    
    for geom in gdf.geometry:
        # Handle both Polygon and MultiPolygon
        if geom.geom_type == 'MultiPolygon':
            areas = [p.area for p in geom.geoms]
            perims = [p.length for p in geom.geoms]
            poly = geom.geoms[0]  # Take largest polygon for other features
        else:
            areas = [geom.area]
            perims = [geom.length]
            poly = geom
    
        # Basic shape features
        area = sum(areas)
        perimeter = sum(perims)
        compactness = (4 * np.pi * area) / (perimeter ** 2)
    
        # Convex hull features
        convex_hull = poly.convex_hull
        hull_area = convex_hull.area
        solidity = area / hull_area if hull_area > 0 else 0
    
        # Bounding box features
        minx, miny, maxx, maxy = poly.bounds
        width